/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */

/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name FROM `Facilities` WHERE membercost = 0

/* Q2: How many facilities do not charge a fee to members? */
4

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */
SELECT facid, name, membercost, monthlymaintenance
FROM `Facilities`
WHERE membercost / monthlymaintenance < 0.2


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */
select * from `Facilities` where facid in (1,5)


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */
select name, monthlymaintenance, case when monthlymaintenance <= 100 then 'cheap'
                                      when monthlymaintenance > 100 then 'expensive'
                                      end AS category from `Facilities`


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */
SELECT firstname, surname
FROM `Members`
WHERE joindate
IN (

SELECT MAX( joindate )
FROM `Members`
)

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */
SELECT DISTINCT CONCAT( m.firstname, ' ', m.surname ) AS membername, f.name
FROM `Members` m, `Bookings` b, `Facilities` f
WHERE m.memid = b.memid
AND b.facid = f.facid
AND f.facid
IN ( 0, 1 )


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */
SELECT f.name as facility,CASE WHEN b.memid > 0 THEN f.membercost
                   WHEN b.memid = 0 THEN f.guestcost
                   END AS cost,
       CONCAT(m.firstname,' ',m.surname) AS membername
       FROM `Members` m, `Bookings` b, `Facilities` f
       WHERE (CASE WHEN b.memid > 0 THEN f.membercost
                   WHEN b.memid = 0 THEN f.guestcost
                   END) > 30
              AND b.starttime LIKE '2012-09-14%' AND b.facid = f.facid AND b.memid = m.memid
       ORDER BY cost DESC

/* Q9: This time, produce the same result as in Q8, but using a subquery. */
SELECT bfm.facility, bfm.cost, bfm.membername
FROM (SELECT f.name as facility,CASE WHEN b.memid > 0 THEN f.membercost
                   WHEN b.memid = 0 THEN f.guestcost
                   END AS cost,
       CONCAT(m.firstname,' ',m.surname) AS membername
       FROM `Members` m, `Bookings` b, `Facilities` f
       WHERE b.starttime LIKE '2012-09-14%' AND b.facid = f.facid AND b.memid = m.memid) as bfm       
WHERE  bfm.cost > 30
        
ORDER BY cost DESC

/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [1]:
import sqlite3 
con = sqlite3.connect('/Users/lestinjackson/dev/DataScienceGuidedCapstone/Notebooks/sqlite_db_pythonsqlite.db')

cursorObj= con.execute("SELECT f.name, sum(CASE WHEN b.memid = 0 THEN guestcost WHEN b.memid > 0 THEN membercost END) "+
                       " AS totalcost FROM `Facilities` f, `Bookings` b "+
                       "WHERE f.facid = b. facid GROUP BY f.name HAVING totalcost < 1000 ORDER BY totalcost LIMIT 30")
for row in cursorObj:
  print ( row , "\n")

('Table Tennis', 90) 

('Snooker Table', 115) 

('Pool Table', 265) 

('Badminton Court', 604.5) 



/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [2]:
cursorObj = con.execute("SELECT m1.surname, m1.firstname, m2.surname||m2.firstname AS recommend "+
                        "FROM `Members` m1, "+ 
" `Members` m2 WHERE m1.recommendedby = m2.memid "+
"ORDER by m1.surname, m1.firstname LIMIT 30")
for row in cursorObj:
  print ( row , "\n")

('Bader', 'Florence', 'StibbonsPonder') 

('Baker', 'Anne', 'StibbonsPonder') 

('Baker', 'Timothy', 'FarrellJemima') 

('Boothe', 'Tim', 'RownamTim') 

('Butters', 'Gerald', 'SmithDarren') 

('Coplin', 'Joan', 'BakerTimothy') 

('Crumpet', 'Erica', 'SmithTracy') 

('Dare', 'Nancy', 'JopletteJanice') 

('Genting', 'Matthew', 'ButtersGerald') 

('Hunt', 'John', 'PurviewMillicent') 

('Jones', 'David', 'JopletteJanice') 

('Jones', 'Douglas', 'JonesDavid') 

('Joplette', 'Janice', 'SmithDarren') 

('Mackenzie', 'Anna', 'SmithDarren') 

('Owen', 'Charles', 'SmithDarren') 

('Pinker', 'David', 'FarrellJemima') 

('Purview', 'Millicent', 'SmithTracy') 

('Rumney', 'Henrietta', 'GentingMatthew') 

('Sarwin', 'Ramnaresh', 'BaderFlorence') 

('Smith', 'Jack', 'SmithDarren') 

('Stibbons', 'Ponder', 'TracyBurton') 

('Worthington-Smyth', 'Henry', 'SmithTracy') 



/* Q12: Find the facilities with their usage by member, but not guests */

In [3]:
cursorObj = con.execute("SELECT f.name, sum(CASE WHEN b.memid = 0 THEN 0 WHEN b.memid > 0 THEN membercost END) AS totalcost "+ 
"FROM `Facilities` f, `Bookings` b WHERE f.facid = b. facid "+
"GROUP BY f.name  ORDER BY totalcost LIMIT 30")
for row in cursorObj:
  print ( row , "\n")

('Badminton Court', 0) 

('Pool Table', 0) 

('Snooker Table', 0) 

('Table Tennis', 0) 

('Massage Room 2', 267.30000000000007) 

('Squash Court', 682.5) 

('Tennis Court 2', 1380) 

('Tennis Court 1', 1540) 

('Massage Room 1', 4167.900000000023) 



/* Q13: Find the facilities usage by month, but not guests */

In [4]:
cursorObj = con.execute("SELECT f.name, SUBSTR(b.starttime,1, 7) AS MONTH, sum(CASE WHEN b.memid = 0 "+
                        "THEN guestcost WHEN b.memid > 0 THEN membercost END) AS totalcost "+
"FROM `Facilities` f, `Bookings` b WHERE f.facid = b. facid " +
"GROUP BY f.name, MONTH  ORDER BY f.name, MONTH LIMIT 30")
for row in cursorObj:
  print ( row , "\n")

('Badminton Court', '2012-07', 77.5) 

('Badminton Court', '2012-08', 217.0) 

('Badminton Court', '2012-09', 310.0) 

('Massage Room 1', '2012-07', 4442.300000000002) 

('Massage Room 1', '2012-08', 7194.699999999983) 

('Massage Room 1', '2012-09', 9170.899999999967) 

('Massage Room 2', '2012-07', 679.5999999999999) 

('Massage Room 2', '2012-08', 2569.1000000000004) 

('Massage Room 2', '2012-09', 3738.600000000001) 

('Pool Table', '2012-07', 35) 

('Pool Table', '2012-08', 95) 

('Pool Table', '2012-09', 135) 

('Snooker Table', '2012-07', 35) 

('Snooker Table', '2012-08', 25) 

('Snooker Table', '2012-09', 55) 

('Squash Court', '2012-07', 990.5) 

('Squash Court', '2012-08', 1785.0) 

('Squash Court', '2012-09', 2194.5) 

('Table Tennis', '2012-07', 15) 

('Table Tennis', '2012-08', 20) 

('Table Tennis', '2012-09', 55) 

('Tennis Court 1', '2012-07', 900) 

('Tennis Court 1', '2012-08', 1430) 

('Tennis Court 1', '2012-09', 1710) 

('Tennis Court 2', '2012-07', 880) 

('Tenni